In [1]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

# بارگذاری داده‌ها
answer = pd.read_csv("C:\\Users\\sepeh\\Downloads\\titanic\\gender_submission.csv")
answer = answer.dropna(subset=['Survived'])
print(len(answer))
test = pd.read_csv("C:\\Users\\sepeh\\Downloads\\titanic\\test.csv")
train = pd.read_csv("C:\\Users\\sepeh\\Downloads\\titanic\\train.csv")
# حذف ردیف‌های نامعتبر
train2 = train.dropna(subset=[   'Pclass', 'Sex', 'Survived'], inplace=False)
test2 = test.dropna(subset=['Sex', 'Pclass'], inplace=False)  # مطمئن شوید که داده‌های تست نیز کامل هستند
test_ar = np.array(test2[['PassengerId',		'Pclass',	'Name',	'Sex',	'Age', 	'SibSp', 	'Parch'	,'Ticket'	,'Fare'	,'Cabin',	'Embarked'
]])
train_ar = np.array(train2[['PassengerId',		'Pclass',	'Name',	'Sex',	'Age', 	'SibSp', 	'Parch'	,'Ticket'	,'Fare'	,'Cabin',	'Embarked'
]])
data = np.concatenate((train_ar, test_ar), axis = 0)
data = pd.DataFrame(data, columns=
['PassengerId', 'Pclass','Name', 'Sex',	'Age', 	'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'
])
# data  = data.dropna (subset = ['PassengerId',		'Pclass',	'Name',	'Sex',	'Age', 	'SibSp', 	'Parch'	,'Ticket'	,'Fare'	,'Cabin',	'Embarked'
# ])
# train = train.dropna(subset = ['PassengerId',		'Pclass',	'Name',	'Sex',	'Age', 	'SibSp', 	'Parch'	,'Ticket'	,'Fare'	,'Cabin',	'Embarked'
# ])
print(len(data))
print(len(test2))
print(len(train2))
print(len(train2["Survived"]))
print("----------------------------")

print(len(data[["PassengerId", "Pclass", "Age", "Sex", "SibSp", "Parch", "Fare", "Embarked"]].iloc[:891]))
print( len(train2['Survived']))
# شناسایی مقادیر مجاز از داده‌های آموزش
# valid_cabins = set(train['Cabin'].unique())
# test = test[test['Cabin'].isin(valid_cabins)]


418
1309
418
891
891
----------------------------
891
891


In [2]:

# تبدیل مقادیر رشته‌ای به عددی
encoder_sex = LabelEncoder()
encoder_Embarked = LabelEncoder()
encoder_Cabin = LabelEncoder()

data['Sex'] = encoder_sex.fit_transform(data['Sex'])
# test['Sex'] = encoder_sex.transform(test['Sex'])  # استفاده از transform برای داده‌های تست

data['Embarked'] = encoder_Embarked.fit_transform(data['Embarked'])
# test['Embarked'] = encoder_Embarked.transform(test['Embarked'])

data['Cabin'] = encoder_Cabin.fit_transform(data['Cabin'])
# test['Cabin'] = encoder_Cabin.transform(test['Cabin'])



In [3]:
# آموزش مدل
# model = DecisionTreeClassifier(max_features=10,criterion='gini', min_samples_split=2 , ccp_alpha=1e-5, min_samples_leaf=8,  max_leaf_nodes=31, max_depth=32, random_state=1)
model = RandomForestClassifier(n_estimators=200, criterion='gini', max_features=8, min_samples_split=2, min_samples_leaf=2, max_leaf_nodes=32)
model.fit(data[["PassengerId", "Pclass", "Age", "Sex", "Parch", "Fare", "Embarked"]].iloc[:891], train2['Survived'])

# پیش‌بینی
pred = model.predict(data[["PassengerId", "Pclass", "Age", "Sex", "Parch", "Fare", "Embarked"]].iloc[:891])
report = classification_report(train2['Survived'], pred)
print(report)

pred_test = model.predict(data[["PassengerId", "Pclass", "Age", "Sex", "Parch", "Fare", "Embarked"]].iloc[891:1309])
print(len(pred_test))

print(f"Length of answer: {len(answer['Survived'])}, Length of predictions: {len(pred_test)}")

# گزارش
print(classification_report(answer['Survived'], pred_test))

              precision    recall  f1-score   support

           0       0.89      0.96      0.92       549
           1       0.92      0.81      0.86       342

    accuracy                           0.90       891
   macro avg       0.90      0.88      0.89       891
weighted avg       0.90      0.90      0.90       891

418
Length of answer: 418, Length of predictions: 418
              precision    recall  f1-score   support

           0       0.89      0.92      0.91       266
           1       0.86      0.80      0.83       152

    accuracy                           0.88       418
   macro avg       0.87      0.86      0.87       418
weighted avg       0.88      0.88      0.88       418



In [4]:

# ایجاد DataFrame نهایی با PassengerId و پیش‌بینی‌ها
passenger_ids_test = data["PassengerId"].iloc[891:1309]  # گرفتن PassengerId برای داده‌های تست
predictions_df = pd.DataFrame({
    'PassengerId': passenger_ids_test,
    'Survived': pred_test
})
predictions_df.to_csv("predict_Survived2.csv",index=False)
print(predictions_df.head())

    PassengerId  Survived
891         892         0
892         893         0
893         894         0
894         895         0
895         896         1


In [5]:
print(classification_report(answer['Survived'], pred_test))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91       266
           1       0.86      0.80      0.83       152

    accuracy                           0.88       418
   macro avg       0.87      0.86      0.87       418
weighted avg       0.88      0.88      0.88       418

